In [1]:
import cirq
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def make_qft_inverse(q):
    for x in range(len(q)):
        yield cirq.H(q[x])
        #add cz power gates connecting this qubit to each of lower ones
        for y in range(len(q) - x - 1):
            exp = -1 * .5 ** (y + 1) #the gate powers are exponents of 1/2 (negative for inv)
            yield cirq.CZ(q[y + x + 1],q[x]) ** exp

In [3]:
def phase_estimation(theta, n_bits, n_reps=10, u_prep_gate=cirq.X):
    circuit = cirq.Circuit()
    qbit = cirq.LineQubit.range(n_bits)
    ubit = cirq.NamedQubit("U")
    #Phase applying gate
    U = cirq.Z ** (2 * theta)#gate
    
    #Prepare state psi
    circuit.append(u_prep_gate(ubit))
    
    #Apply Hadamards
    for q in qbit:
        circuit.append(cirq.H(q))
    
    #Apply controlled U operations
    for x in range(n_bits):
        power = 2 ** (n_bits - x - 1) # 2^(n-1), 2^n-2, 2^n-3 ...
        circuit.append(cirq.ControlledGate(U).on(qbit[x], ubit) ** power)
        
    #Inverse QFT
    circuit.append(make_qft_inverse(qbit))
    
    #Measure
    circuit.append(cirq.measure(*qbit, key='m'))
    
    #Simulate
    sim = cirq.Simulator()
    result = sim.run(circuit, repetitions=n_reps)
    theta_estimate = np.sum(2 ** np.arange(n_bits) * result.measurements['m'], axis=1) / 2**n_bits
    
    return theta_estimate

In [ ]:
#Test
theta = np.pi/4
n = 25
result = phase_estimation(theta=theta, n_bits=n, n_reps=20, u_prep_gate=cirq.X)
result *= 4
print(np.mean(result))

